In [1]:
import cv2 as cv
print(cv.__version__)

4.4.0


In [17]:
img_test = cv.imread('C:/Users/Ryu/Desktop/CTSkinSegmentation/CTSkinSegmentation/test.png')
print(type(img_test))
print(img_test.shape)
print(img_test[90:,90:,0])

print()

img_cvtGray = cv.cvtColor(img_test, cv.COLOR_BGR2GRAY)
print(img_cvtGray[90:,90:])
print(img_cvtGray.shape)

<class 'numpy.ndarray'>
(180, 279, 3)
[[ 45  45  45 ... 255 255 255]
 [ 45  45  45 ... 255 255 255]
 [ 45  45  45 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]

[[ 46  46  46 ... 255 255 255]
 [ 46  46  46 ... 255 255 255]
 [ 46  46  46 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
(180, 279)


In [3]:
# 얕은 복사(같은 id) vs 깊은 복사(다른 id)

# 1. 얕은 복사
a = [1,2,3]
b = a
print(id(a), id(b)) # 같은 id (같은 주소값)

b[0] = 5000
print(a, b) # 같은 곳을 가리키므로, b를 바꿔도 a값까지 바뀜!

print()

# 2. 깊은 복사
c = [1,2,3]
d = c.copy()
print(id(c), id(d))

d[0] = 5000
print(c, d)

2900486167688 2900486167688
[5000, 2, 3] [5000, 2, 3]

2900486167880 2900486167624
[1, 2, 3] [5000, 2, 3]


### 함수로 배열 넘기기
함수로 배열을 넘기는 행위가 배열 자체를 전달해서 새로운 배열로 복사하는 것이 아니라, **"주소값"**을 넘겨주는 것이다!

In [71]:
def test(arr):
    print(id(arr))

arr_test = np.ones(10)
print(id(arr_test))
test(arr_test)    

2899327452624
2899327452624


### cv2.equalizeHist(ds.pixel_array)
The function equalizeHist is histogram equalization of images and only implemented for CV_8UC1 type, which is a single channel 8 bit unsigned integral type.  
> 따라서 원본이 1000 넘는 값들로 이루어져 uint8 (8bit, 256)이 아니므로, 이 opencv함수 말고 직접 코딩해야됨... (for improve)  
>  ==> **map_uint16_to_uint8() 함수!**

### cf. fail modules
* ArrayDicom = np.zeros(shape, dtype=np.uint8)
* cv.equalizeHist()
* cv2.createCLAHE()
* cv.normalize()
* skimage.util.img_as_ubyte() (from skimage import ima_as_ubyte) 

In [93]:
import cv2 as cv
import glob
import pydicom as dicom
import os
import numpy as np

def Overlay(back, front, rows, cols):
    # array indexing using np.where ==> 2중 for문보다 속도 훨씬 빠름!
    back[:, :, 0] = np.where(front != 0, back[:, :, 0] * 0.7 + 255 * 0, 0)
    back[:, :, 1] = np.where(front != 0, back[:, :, 1] * 0.7 + 255 * 0, 0)
    back[:, :, 2] = np.where(front != 0, back[:, :, 2] * 0.7 + 255 * 0.3, 0)
    #print(back.shape) # (512, 512, 3) ==> (열, 행, 면)
    
    '''
    for i in range(rows):
        for j in range(cols):
            if (front[i][j] == 0) : continue
            
            # linear interpolation
            back[i][j][0] = back[i][j][0] * 0.7 + 255 * 0 # b
            back[i][j][1] = back[i][j][1] * 0.7 + 255 * 0 # g
            back[i][j][2] = back[i][j][2] * 0.7 + 255 * 0.3 # r
    '''

# from 0 ~ 4095 (16bit, uint16) to 0 ~ 255 (8bit, uint8)
def map_uint16_to_uint8(img, lower_bound=None, upper_bound=None):
    lower_bound = np.min(img)
    upper_bound = np.max(img)
    
    if not(0 <= lower_bound < 2**16) and lower_bound is not None:
        raise ValueError(
            '"lower_bound" must be in the range [0, 65535]')
    if not(0 <= upper_bound < 2**16) and upper_bound is not None:
        raise ValueError(
            '"upper_bound" must be in the range [0, 65535]')
    if lower_bound is None:
        lower_bound = np.min(img)
    if upper_bound is None:
        upper_bound = np.max(img)
    if lower_bound >= upper_bound:
        raise ValueError(
            '"lower_bound" must be smaller than "upper_bound"')
    lut = np.concatenate([
        np.zeros(lower_bound, dtype=np.uint16),
        np.linspace(0, 255, upper_bound - lower_bound).astype(np.uint16),
        np.ones(2**16 - upper_bound, dtype=np.uint16) * 255
    ])
    return lut[img].astype(np.uint8)


'''
def DcmToMat(ori):
    return map_uint16_to_uint8(ori.pixel_array)
'''

def main():
    files_str = [file for file in glob.glob("C:/Users/Ryu/Desktop/180509_SampleData_CT/*.dcm")] # imread(dcm) : None
    
    for f in files_str:
        print(f)
        ori = dicom.read_file(f) # get dicom file / type : <class 'pydicom.dataset.FileDataset'> / size : (512, 512)       
        rows, cols = ori.Rows, ori.Columns
        
        ori1 = map_uint16_to_uint8(ori.pixel_array) # type : np.ndarray ==> cv.imread img도 np.ndarray형!
        
        img_copy = ori1.copy()
        #cv.imwrite('C:/Users/Ryu/Desktop/180509_SampleData_CT/result_python/0. img_copy.png', img_copy) 
        
        
        # 1. Bilateral Filtering (noise filtering)
        filtered = cv.bilateralFilter(img_copy, -1, 15, 15)
        #cv.imwrite('C:/Users/Ryu/Desktop/180509_SampleData_CT/result_python/1. filtered.png', filtered) 
        
        
        # 2. Otsu;s Thresholding (cv2.threshold(src, 0(**T not defined**), 255(change to 255), flag(binary & otsu)))
        ret,otsu = cv.threshold(filtered, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU) # 히스토그램 분석 후 자동으로 T값(ret) 구해서 적용
        #cv.imwrite('C:/Users/Ryu/Desktop/180509_SampleData_CT/result_python/2. otsu.png', otsu) # ex) ret = 33.0
        
        
        # 3. Morphology_preprocessing(remove outlines with erode) ==> anchor(point) default : (-1, -1)
        mask = cv.getStructuringElement(cv.MORPH_RECT,(3,3))
        pre_erode = cv.erode(otsu, mask, iterations = 7)
        #cv.imwrite('C:/Users/Ryu/Desktop/180509_SampleData_CT/result_python/3-1. pre_erode.png', pre_erode)
        pre_dilate = cv.dilate(pre_erode, mask, iterations = 7)
        #cv.imwrite('C:/Users/Ryu/Desktop/180509_SampleData_CT/result_python/3-2. pre_dilate.png', pre_dilate)
        
        
        # 4. Floodfill (combine background to select hole in body) ==> mask : None (not used)
        hole = pre_dilate.copy()
        cv.floodFill(hole, None, (0, 0), 255)
        cv.floodFill(hole, None, (cols-1, rows-1), 255)
        #cv.imwrite('C:/Users/Ryu/Desktop/180509_SampleData_CT/result_python/4. hole.png', hole)
    
    
        # 5. Invert hole
        hole_inv = cv.bitwise_not(hole)
        #cv.imwrite('C:/Users/Ryu/Desktop/180509_SampleData_CT/result_python/5. hole_inv.png', hole_inv)
        
        
        # 6. bitwise OR (combine pre(bone) and hole)
        bitor = np.bitwise_or(pre_dilate, hole_inv)
        #cv.imwrite('C:/Users/Ryu/Desktop/180509_SampleData_CT/result_python/6. bitor.png', bitor)
        
        
        # 7. overlay
        back3C = cv.cvtColor(ori1, cv.COLOR_GRAY2BGR) # 3channel
        front = bitor.copy()
        Overlay(back3C, front, rows, cols)
        
        
        result_path = "C:/Users/Ryu/Desktop/180509_SampleData_CT/result_python/"
        # f == "C:/Users/Ryu/Desktop/180509_SampleData_CT/CT0002.dcm"
        cv.imwrite(result_path + f[-10:-4] + '.png', back3C)
        
        
    
    
main()

C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0002.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0003.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0004.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0005.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0006.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0007.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0008.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0009.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0010.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0011.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0012.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0013.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0014.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0015.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0016.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0017.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0018.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0019.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT00

C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0157.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0158.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0159.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0160.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0161.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0162.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0163.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0164.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0165.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0166.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0167.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0168.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0169.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0170.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0171.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0172.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0173.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT0174.dcm
C:/Users/Ryu/Desktop/180509_SampleData_CT\CT01